# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Find a similar neighborhood <a name="introduction"></a>

In this project we will try to find an similar **neighborhood** bewteen Two cities, **Toronto** and **New York**. 

Here is the scenario. James now is working in Toronto for a IT company and living in Neighborhood **Woburn** which in Borough **Scarborough**. He received another opportunity for a company locates on New York. He decided to accept this opportunity. Before he moves to New York, he wants to find a house with a neighborhood similar to which he is living in the Toronto. 

We will compare the neighborhood with their **Venues** as the first criterion. Then rank the similar neighborhood by the distance from **his new company**. 

We will use our data science powers to generate a most promissing neighborhoods based on these criterias. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* The similarity between different neighborhood according to its venues.
* Distance of neighborhood from his new company.

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* Boroughs list of **Toronto** and **New York** from web(New York: https://cocl.us/new_york_dataset / Toronto:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.)
* Same with above, we will obtain the neighborhood list from the web.
* Longitude and latitude will be obtained by using **Google Maps API reverse geocoding**
* Venues  will be obtained using **Foursquare API**

### Prepare the Neighborhood data for New York city

In [35]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import codecs
from urllib import request
import requests 
from bs4 import BeautifulSoup
import re

print('Libraries imported.')

Libraries imported.


In [27]:
!wget -q -O newyork_data.json https://cocl.us/new_york_dataset #if you r running in linux, please use single quotes to mark the file name
print('Data downloaded!')

Data downloaded!


In [36]:
with open('newyork_data.json','r') as json_data:
    #print(json_data)
    newyork_data = json.load(json_data)

In [37]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [38]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neighborhoods = pd.DataFrame(columns=column_names)

#Then let's loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhoods = ny_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### let's show the data frame

In [39]:
ny_neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


### Prepare Neighborhood the data for Toronto

#### Use the BeatuifulSoup to grab the table from website

In [41]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = request.Request(URL, headers={'User-Agent': 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'})
page = request.urlopen(req)

soup = BeautifulSoup(page, "html.parser")
page.close()
tables = soup.findAll('table')
tab = tables[0]
#print(tables)

col = []
for th in tab.tbody.findAll('th'):
    text = th.getText()
    text = re.sub(r'\n','',text)
    col.append(text)
toronto_pd = pd.DataFrame(columns = col)
df_index = 0
for tr in tab.tbody.findAll('tr'):
    row = []
    for td in tr.findAll('td'):        
        text = td.getText()
        text = re.sub(r'\n','',text)
        row.append(text)
        #print(text)
    if len(row) != 0: 
        #print(row)
        toronto_pd.loc[df_index] =row
        df_index = df_index + 1

In [42]:
    #let's check the data
    toronto_pd.columns = ['Postcode', 'Borough', 'Neighborhood']
    toronto_pd.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


But there is a lot invalid Borough and "Not assigned". Let's get the valid Borough value list and remove the "Not assigned" in borough

In [43]:

valid_borough  = list(toronto_pd.Borough)
valid_borough_set = set(valid_borough)
#print(valid_borough_set)
valid_borough_set.remove('Not assigned')
#print(valid_borough_set)
toronto_pd = toronto_pd[toronto_pd.Borough.isin(list(valid_borough_set))]
#reset the index
toronto_pd = toronto_pd.reset_index(drop = True)


Use the borough name instead of the neighborhood name if it is "Not assigned"

In [44]:
print(len(toronto_pd))
for i in range(0, len(toronto_pd)):
    print#i)
    toronto_pd.iloc[i]['Neighborhood']
    if toronto_pd.iloc[i]["Neighborhood"] == "Not assigned":
        toronto_pd.iloc[i]['Neighborhood'] = toronto_pd.iloc[i]['Borough']
toronto_pd.head(10)

210


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Combine the Neighborhood name if the Postcode and Borough is same and show the final table

In [45]:
toronto_pd = toronto_pd.groupby(['Postcode','Borough'],as_index=False).agg(','.join)
toronto_pd.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Then let's combine the coordinates and Postcode from a csv.

In [46]:
!wget --quiet http://cocl.us/Geospatial_data -O Torontocoordinates.csv

In [47]:
toronto_coordinates = pd.read_csv('Torontocoordinates.csv',skiprows = 1,header = None,names  = ["Postcode","Latitude","Longitude"])
postcode_list = list(toronto_pd.Postcode)
toronto_coordinates = toronto_coordinates[toronto_coordinates.Postcode.isin((postcode_list))]

In [48]:
final_pd = pd.merge(toronto_pd, toronto_coordinates, how='outer', on=['Postcode'])
final_pd.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


Above it the final Data for Toronto, include the Neighborhood and coordinates.

### Grab the venues from the Foursquare API

Now we already have the coordinates data for every single Neighborhood.Next we can use the coordinates information to grab the list of venues that most currently checked in. We consider that the venues of a neighborhood is the reason why that people will decide to live there. Then if two neighborhoods have similar veneus, we can consider they are similar neighborhood.

In [71]:
CLIENT_ID = 'OLGVOKCOGQNJ20LIXXMEHXP102OYOT4VMF0FVD42T4HGASN1' # your Foursquare ID
CLIENT_SECRET = 'MJKOST5555V43XDDM0ZDLFNAY2ONZVOC1Q4ZE4CJG2RB4TVV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: OLGVOKCOGQNJ20LIXXMEHXP102OYOT4VMF0FVD42T4HGASN1
CLIENT_SECRET:MJKOST5555V43XDDM0ZDLFNAY2ONZVOC1Q4ZE4CJG2RB4TVV


In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronto_venues = getNearbyVenues(names=final_pd['Neighborhood'],
                                   latitudes=final_pd['Latitude'],
                                   longitudes=final_pd['Longitude']
                                  )
ny_venues = getNearbyVenues(names=ny_neighborhoods['Neighborhood'],
                                   latitudes=ny_neighborhoods['Latitude'],
                                   longitudes=ny_neighborhoods['Longitude']
                                  )

43.806686299999996
-79.19435340000001
Rouge,Malvern
43.7845351
-79.16049709999999
Highland Creek,Rouge Hill,Port Union
43.7635726
-79.1887115
Guildwood,Morningside,West Hill
43.7709921
-79.21691740000001
Woburn
43.773136
-79.23947609999999
Cedarbrae
43.7447342
-79.23947609999999
Scarborough Village
43.7279292
-79.26202940000002
East Birchmount Park,Ionview,Kennedy Park
43.711111700000004
-79.2845772
Clairlea,Golden Mile,Oakridge
43.716316
-79.23947609999999
Cliffcrest,Cliffside,Scarborough Village West
43.692657000000004
-79.2648481
Birch Cliff,Cliffside West
43.7574096
-79.27330400000001
Dorset Park,Scarborough Town Centre,Wexford Heights
43.750071500000004
-79.2958491
Maryvale,Wexford
43.7942003
-79.26202940000002
Agincourt
43.7816375
-79.3043021
Clarks Corners,Sullivan,Tam O'Shanter
43.8152522
-79.2845772
Agincourt North,L'Amoreaux East,Milliken,Steeles East
43.799525200000005
-79.3183887
L'Amoreaux West
43.836124700000006
-79.20563609999999
Upper Rouge
43.8037622
-79.3634517
Hillcr

40.64510294925429
-74.01031618527784
Sunset Park
40.7302009848647
-73.95424093127393
Greenpoint
40.59526001306593
-73.97347087708445
Gravesend
40.57682506566604
-73.96509448785336
Brighton Beach
40.58689012678384
-73.94318640482979
Sheepshead Bay
40.61443251335098
-73.95743840559939
Manhattan Terrace
40.63632589026677
-73.95840106533903
Flatbush
40.67082917695294
-73.94329119073582
Crown Heights
40.64171776668961
-73.93610256185836
East Flatbush
40.642381958003526
-73.98042110559474
Kensington
40.65694583575104
-73.98007340430172
Windsor Terrace
40.676822262254724
-73.9648592426269
Prospect Heights
40.66394994339755
-73.91023536176607
Brownsville
40.70714439344251
-73.95811529220927
Williamsburg
40.69811611017901
-73.92525797487045
Bushwick
40.687231607720456
-73.94178488690297
Bedford Stuyvesant
40.695863722724084
-73.99378225496424
Brooklyn Heights
40.687919722485574
-73.99856139218463
Cobble Hill
40.680540231076485
-73.99465372828006
Carroll Gardens
40.676253230250886
-74.0127589747

40.542230747450745
-74.1643308041936
Eltingville
40.53811417474507
-74.17854866165878
Annadale
40.541967622888755
-74.20524582480326
Woodrow
40.50533376115642
-74.24656934235283
Tottenville
40.637316067110326
-74.08055351790115
Tompkinsville
40.61919310792676
-74.09629029235458
Silver Lake
40.61276015756489
-74.0971255217853
Sunnyside
40.643675183340974
-73.96101312466779
Ditmas Park
40.66094656188111
-73.93718680559314
Wingate
40.655572313280764
-73.92688212616955
Rugby
40.60919044434558
-74.08015734936296
Park Hill
40.62109047275409
-74.13304143951704
Westerleigh
40.620171512231884
-74.15315246387762
Graniteville
40.63532509911492
-74.16510420241124
Arlington
40.596312571276734
-74.06712363225574
Arrochar
40.59826835959991
-74.0766743627905
Grasmere
40.59632891379513
-74.08751118005578
Old Town
40.588672948199275
-74.09639905312521
Dongan Hills
40.57352690574283
-74.09348266303591
Midland Beach
40.57621558711788
-74.10585598545434
Grant City
40.56425549307335
-74.10432707469124
New D

**Let's do the one hot encode and groupby the Neighborhood name.**

In [82]:
# one hot encede and calculate the nubmers of venes of one category in one niieghborhood.
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot = toronto_onehot.drop(['Neighborhood'], axis=1)
#toronto_onehot.Neighborhood
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#print(toronto_onehot.columns[-1] )
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,2,0,0,1,0,3,0,0,0,0,0,3,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,3,0,0,0,1,3,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,1,0,0,2,0,0,3,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,

In [83]:
# one hot encede and calculate the nubmers of venes of one category in one niieghborhood.
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot = ny_onehot.drop(['Neighborhood'], axis=1)
#toronto_onehot.Neighborhood
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

#print(toronto_onehot.columns[-1] )
# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_grouped = ny_onehot.groupby('Neighborhood').sum().reset_index()
ny_grouped.head(10)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Offic

**Now we got all the data we need for our target.** Next we will introduce the Methodology of how to find a similar Neighborhood.